In [ ]:
import logging

import geopandas as gpd
import hvplot.pandas
import pandas as pd

from searvey._ndbc_api import fetch_ndbc_stations_data,get_ndbc_stations
logging.basicConfig(
    level=20,
    style="{",
    format="{asctime:s}; {levelname:8s}; {threadName:23s}; {name:<25s} {lineno:5d}; {message:s}",
)

logging.getLogger("urllib3").setLevel(30)
logging.getLogger("parso").setLevel(30)

logger = logging.getLogger(__name__)

## Retrieve all NDBC stations

In [ ]:
ndbc_stations = get_ndbc_stations()
ndbc_stations

## Visualize the location of the stations on a map

In [ ]:
# Create the plot
def plot_map(data, title):

    # Plot the world map
    world_plot = data.hvplot(geo=True, tiles=True, hover_cols=["Station", "location"])
    return world_plot.opts(width=800, height=500)


plot_map(ndbc_stations, 'NDBC Stations Worldwide')

## Get the stations from a specific region and vizulaise it in a map

In [ ]:
from shapely.geometry import box
# Define a rectangular region
region = box(-85, 25, -65, 45)  # Longitude range, Latitude range

# Get stations within the region
east_coast_stations = get_ndbc_stations(region=region)
plot_map(east_coast_stations, 'NDBC Stations on the East Coast of the US')

Not all stations provide the same set of measurements, the following measurements are listed and which type of stations support them:

```markdown

| Package Name          | C-MAN  | Moored |
|-----------------------|--------|--------|
| adcp                  | [ ]    | [ ]    |
| cwind                 | [X]    | [ ]    |
| ocean                 | [ ]    | [ ]    |
| spec                  | [ ]    | [ ]    |
| stdmet                | [X]    | [ ]    |
| supl                  | [ ]    | [X]    |
| swden                 | [ ]    | [X]    |
| swdir                 | [ ]    | [X]    |
| swdir2                | [ ]    | [X]    |
| swr1                  | [ ]    | [X]    |
| swr2                  | [ ]    | [X]    |
```


(note: stations that are with only numbers are usually Moored Buoyse e.g. 44008 and stations with a mixture of numbers and letters are C-MAN owned e.g. BUZM3)
C-MAN -> Numbers only
Moored Buoyse -> numbers and letters

The table was tested at the start of june 2024 and may differ as support for different stations status may change.

Note: If an error is logged,it may be because one of the stations does not have the data of the mode


## Retrieve Data for Selected Stations

In [ ]:
# Define start and end dates for data retrieval
start_date = "2023-01-01"
end_date = "2023-01-10"

# Get data for selected stations (mode = 'stdmet' for standard meteorological data)
station_ids = east_coast_stations["Station"].tolist()[1:3]
data_df = fetch_ndbc_stations_data(
    station_ids=["SRST2","VENF1"],
    mode="stdmet",
    start_date=start_date,
    end_date=end_date,
)

#output is a dictionary of {station id : data}
data_to_plot = data_df["SRST2"]
data_to_plot

## Plot the data of the dataframes

In [ ]:
# Plot the data of data_to_plot
data_to_plot[["WSPD", "GST", "ATMP"]].rename(columns={"WSPD": "Wind Speed", "GST": "Wind Gust", "ATMP": "Air Temperature"}).hvplot(grid=True)